### GIL: Global interpreter lock.

> We cannot run two threads in one process at the same time. If we have our Python process create another thread, the main thread and that other thread are not going to be able to run at the same time, even if we did have two or more cores. <br> <br> That's because each process in Python creates a key `resource`, a critical `resource`, and when a thread is running it must acquire that `resource`. And every process creates only one of these. <br> <br> The process creates this unique `resource` and when a thread is running it must acquire this `resource`. Python will check that the thread has that `resource` before it runs it. Because there's only one of those `resource`s, we can only run one thread in that process at once.

This resouce is GIL. That's the resource that the process creates and the threads must acquire. This is what the process creates, this Global Interpreter Lock. A thread must acquire it then they can run, and then they must release it for another thread to run. So we cannot run two threads at the same time.

*So what is the point of threads? If they can not run at the same time, and also they can make our code slower.*

The point of threads in Python is - to reduce waiting time. We cannot run two threads at the same time in Python under the same process.

#### Blocking statements:

This function here:

```python
user_input = input('Enter your name: ')
```
 is going to wait for the user to type something. This is called a blocking operation, an operation where the thread is blocked, waiting for something to happen. These operations are what makes our threaded code slow.

In [28]:
from threading import Thread, Timer
import random
import time
import queue

In [3]:
def timed_function():
    print("""
    This function, even though started first, 
    will print only after timeout and succeeding statements 
    won't wait for it to complete")
    """)
def ask_user():
    start = time.time()
    user_input = input('Enter your name: ')
    greet = f'Hello, {user_input}'
    print(greet)
    print('ask_user: ', time.time() - start)

def complex_calculation_blocking():
    print('Started calculating...')
    start = time.time()
    time.sleep(5)
    print('complex_calculation: ', time.time() - start)
    
def complex_calculation():
    print('Started calculating...')
    start = time.time()
    timer = Timer(random.randint(a=5, b=10), timed_function)
    timer.start()
    print('complex_calculation: ', time.time() - start)

Whenever we create a thread, that's gonna go to the operating system ask the operating system to give us a new thread.

In [8]:
start = time.time()
ask_user()
complex_calculation_blocking()
print('Single thread total time: ', time.time() - start, '\n\n')

Enter your name: adf
Hello, adf
ask_user:  0.8538475036621094
Started calculating...
complex_calculation:  5.014180898666382
Single thread total time:  5.869045972824097 




### Using threads

In [9]:
# Creating threads. When we actually start the thread, only then it runs the functions.

th1 = Thread(target=complex_calculation_blocking)
th2 = Thread(target=ask_user)

Now we have three threads - a main thread, which is responsible for running through the app, a thread which is responsible for running the `complex_calculation`, and another thread which is responsible for running the `ask_user` function.

In [10]:
start = time.time()

# Starting the threads at same time as main thread.
th1.start()
th2.start()


Started calculating...
Enter your name: asdfdsa
Hello, asdfdsa
ask_user:  1.479612112045288
complex_calculation:  5.0086352825164795


> thread1 runs ask_user, thread2 (worker threads) runs complex_calculation_blocking main thread runs the all the code here. So we need to tell the main thread to wait, and not exit until the two threads finish

In [11]:
th1.join()
th2.join()

Now we have three threads (2 worker, 1 main) running at the same time (concurrently). 

When we start `th1`, the `complex_calculation` function is running. When we start `th2`, the `ask_user` function is running.

And also, the main thread which is responsible for running this code, is also running. 

So we have to tell our main thread to wait for these two threads to finish. The way we do that: `th1.join()` and `th2.join()`. This tells our main thread to wait for thread one to finish and wait for thread two to finish.

When we have a blocking operation like waiting for user input, that is waiting, making the programme wait for something, that's a good use for a thread.

### `ThreadPoolExecutor`

In [1]:
from concurrent.futures import ThreadPoolExecutor

In [5]:
# Does `as` create in instance?

with ThreadPoolExecutor(max_workers=2) as pool:
    pool.submit(complex_calculation_blocking)
    pool.submit(ask_user)

Started calculating...


### Multi processing in python

Due to the way Windows forks processes, we must make sure that the code that starts a process is surrounded by 

`if __name__ == "__main__"`


Otherwise when we start new processes on Windows, those processes automatically start new processes, and those start new ones, and so on. Python will not allow this to happen, and as protection it requires the above if statement.

Replace `process.start()`, with:

```python
if __name__ == "__main__":
    process.start()
    ...
    process.join()
```

It's important that all the code in between starting the process and joining the process is inside the if statement.

### Shared state between threads.

- **Atomic Operation:** An atomic operation is one that cannot be interrupted in the middle of it. So we cannot interrupt an atomic operation halfway through it by changing to a new thread. Whenever we're doing an atomic operation we know that it's going to finish before we can unplug the thread from the core and put another one in. <br> For example, a print statement cannot print only half the line and then be interrupted. Similarly, appending to a deque cannot be interrupted halfway through.

In [25]:
# Shared state between threads
counter = 0

# Adding random sleeps between statements when doing multithreaded codes is called fuzzying,

def incr_counter():
    global counter
    counter += 1
    time.sleep(random.random())
    print(f'New counter: {counter}')

In [27]:
for x in range(10):
    t = Thread(target=incr_counter)
    time.sleep(random.random())
    t.start()

New counter: 12
New counter: 12
New counter: 13
New counter: 14
New counter: 15
New counter: 16
New counter: 17
New counter: 18
New counter: 20
New counter: 20


### Using queue for thread sequencing

In [38]:
job_q = queue.Queue()
count_q = queue.Queue()
counter = 0


def incr_mgr():
    global counter
    
    while True:
        # this waits until an item is available and then locks the queue.
        # and when it is available, it's not going to allow any other threads to get anything until we are done.
        incr = count_q.get()
        old_counter = counter
        counter = old_counter + incr
        job_q.put((f'New value: {counter}', '-------'))
        
        # This unlocks the queue, so that another thread could get something if it wanted.
        count_q.task_done()
        
def print_mgr():
    while True:
        for line in job_q.get():
            print(line)
            
        job_q.task_done()
        
def incr_counter():
    count_q.put(1)

In [39]:
# daemon: true => The thread is going to run until an error is encountered
thr_inc = Thread(target=incr_mgr, daemon=True)
thr_print = Thread(target=print_mgr, daemon=True)
worker_threads = [Thread(target=incr_counter) for thread in range(10)]

thr_inc.start()
thr_print.start()

for i in worker_threads:
    i.start()
    
for i in worker_threads:
    i.join()
    
count_q.join()
job_q.join()

New value: 1
-------
New value: 2
-------
New value: 3
-------
New value: 4
-------
New value: 5
-------
New value: 6
-------
New value: 7
-------
New value: 8
-------
New value: 9
-------
New value: 10
-------


### Generators instead of threads for concurrent processing

Multi-tasking without threads.

> So very similar to a thread, really, if we think about it. A Thread can at any point be suspended or removed from a core and then it can be brought back and it continues running where it's left off. So the generator actually behaves fairly similar to a thread in that when we arrive at a yield we're removing it from the core, and when we execute next, we're sort of bringing it back and it continues running.

In [40]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1
        
g1 = countdown(10)
g2 = countdown(20)

In [41]:
print(next(g1))
print(next(g2))
print(next(g1))
print(next(g2))

10
20
9
19


In [42]:
tasks = [countdown(10), countdown(5), countdown(20)]

while tasks:
    task = tasks[0]
    tasks.remove(task)
    
    try:
        print(next(task))
        tasks.append(task)
        
    except Exception:
        print("Task completed")

10
5
20
9
4
19
8
3
18
7
2
17
6
1
16
5
Task completed
15
4
14
3
13
2
12
1
11
Task completed
10
9
8
7
6
5
4
3
2
1
Task completed


#### Yielding from another iterator.

In [4]:
from collections import deque

In [5]:
models = deque(('Catherine', 'Elizabeth', 'Nicole'))

def get_models():
    yield from models


In [6]:
g = get_models()

In [7]:
next(g)

'Catherine'

In [8]:
def greet_model(generator):
    while True:
        try:
            model = next(generator)
            yield f'Hello {model}'
            
        except StopIteration:
            pass


In [9]:
g = greet_model(get_models())

In [10]:
next(g)

'Hello Catherine'

#### Receive data with yield statement

In [11]:
models = deque(('Catherine', 'Elizabeth', 'Nicole'))

"""
Generators that receive data, they're really no longer called generators, 
because they're not generating anything. 
Now they're receiving data and doing something with it. 
This type of generator is called a co-routine. 

And in Python, they're known as co-routines because they take in data, 
and they can be suspended.
"""
def model_upper():
    while models:
        model = models.popleft().upper()
        greeting = yield
        print(f'{greeting} {model}')
        
        
def greet(gen):
    # yield from g
    gen.send(None)
    
    while True:
        greeting = yield
        gen.send(greeting)
        


In [40]:
greeter = greet(model_upper())

In [41]:
greeter.send(None)

In [42]:
greeter.send('Hello')

Hello CATHERINE


In [43]:
# Pausing the function and doing something else
print("Hello world multi-tasking")

Hello world multi-tasking


In [44]:
# Resuming the function
greeter.send('How are you!')

How are you! ELIZABETH


### Generator based Coroutines

In [2]:
from types import coroutine

In [13]:
async def greet_async(gen):
    await gen
    
@coroutine
def model_upper():
    while models:
        model = models.popleft().upper()
        greeting = yield
        print(f'{greeting} {model}')

In [14]:
greeter = greet_async(model_upper())

In [15]:
greeter.send(None)

In [16]:
greeter.send('Hello')

Hello CATHERINE
